<a href="https://colab.research.google.com/github/rtomek9/plotly-dash-DS4003/blob/main/FINALassignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash

In [2]:
# compiling all code for .py file

import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback
import pandas as pd

df = pd.read_csv('gdp_pcap.csv')
df_long = df.melt(id_vars=df.columns[0],
                  value_vars=df.columns[1:],
                  var_name='year',
                  value_name='GDP')


# initialize app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

# graph
df_long['GDP'] = pd.to_numeric(df_long['GDP'], errors='coerce')
df_long['year'] = pd.to_numeric(df_long['year'], errors='coerce')
df_long = df_long.dropna(subset=['GDP'])
df_long_sorted = df_long.sort_values(by='year')

# define layout and elements
app.layout = html.Div([
    # defines title
    html.H2("GDP per Capita per Country over Years"),
    # defines description
    html.P("The following data is the GDP per Capita of all the selected countries \n"
           "over the course of years 1800-2100. This includes predicted data and past data "
           "collected by the Gap Minder Foundation. \n Select the year from the slider and countries from the dropdown menu."),

    # define drop down with multiple selection and slider
    html.Div([
        html.Div([
            html.Label('Dropdown'),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in df['country']],
                value=['Grit'],
                multi=True
            ),
        ], style={'width': '48%', 'display': 'inline-block', 'margin-right': '2%'}),

        html.Div([
            # define range slider with movable marks that range from 1800 to 2100
            dcc.RangeSlider(
                id='slider-year',
                min=int(df.columns[1]),
                max=int(df.columns[-1]),
                value=[int(df.columns[1]), int(df.columns[-1])],
                # this defines the tooltip or 'Mark' for user-friendly labeling that follows the interaction
                tooltip={
                    "always_visible": True,
                    "template": "{value}"
                },
                marks=None,
                step=None
            ),
        ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'middle'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between', 'margin-top': '50px'}),

    dcc.Graph(
        id = 'graph'
    )
])

@app.callback(
    Output('graph', 'figure'),
    [Input('slider-year', 'value'),
     Input('country-dropdown', 'value')]
)
def update_graph(selected_years, selected_countries):
    filtered_df = df_long_sorted[(df_long_sorted['year'] >= selected_years[0]) & (df_long_sorted['year'] <= selected_years[1])]
    filtered_df = filtered_df[filtered_df['country'].isin(selected_countries)]
    fig = px.line(filtered_df, x='year', y='GDP', color='country', title='GDP per Country by Year')
    fig.update_xaxes(title_text="Year")
    fig.update_yaxes(title_text="GDP")
    return fig

# run app
if __name__ == '__main__':
    app.run(jupyter_mode='inline', debug=True)

<IPython.core.display.Javascript object>